In [1]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from config import OWM_API_key as key, connection_port, mongodb_host
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError
import json
import os
from os import path


API_key = key
host = mongodb_host
port = connection_port
filename = 'resources/zip_list.csv'
global zipcode
owm = OWM(API_key) # the OWM object
global obs
global reception_time
global zid
global zlon
global zlat
global client

In [6]:
def make_zip_list(state):
#     print('using make_zip_list')
    ''' Make a list of zip codes in the specified state.
        Read zip codes from downloadable zip codes csv file available at https://www.unitedstateszipcodes.org/
        
        :param state: the two-letter abreviation of the state whose zip codes you'd like listed
        :type state: string
        
        :returns success_zips: list of zip codes that OWM has records for
        :type success_zips: list
    '''
    import pandas as pd
    from pyowm.exceptions.api_response_error import NotFoundError
    
    successes = 0
    exceptions = 0
    success_zips = []
    fail_zips = []
    
    df = pd.read_csv("resources/zip_code_database.csv")
    # Make a datafram from the rows with the specified state and write the 'zip' column to a list 
    zip_list = df.loc[lambda df: df['state'] == f'{state.upper()}']['zip'].tolist()
#     zip_list = df['zip'].tolist()
    for zipp in zip_list:
        if int(zipp) > 10000:
            try:
#                 print('try setting location for ', zipp)
                set_location(zipp)
#                 print(f'successfully set location for {zipp}', len(success_zips))
                success_zips.append(zipp)
                successes+=1
            except NotFoundError:
                print("except", f'NotFoundError with zipcode {zipp}')
                fail_zips.append(zipp)
                exceptions+=1
                pass
    write_list_to_file(success_zips, 'resources/success_zips.csv')
    write_list_to_file(fail_zips, 'resources/fail_zips.csv')
    print(f'successes = {successes}; exceptions = {exceptions}, all written to files')
    return(success_zips)

def write_list_to_file(zip_list, filename):
#     print('using write_list_to_file')
    """ Write the zip codes to csv file.
        
        :param zip_list: the list created from the zip codes dataframe
        :type zip_list: list stings
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "w") as z_list:
        z_list.write(",".join(str(z) for z in zip_list))
    return
        
def read_list_from_file(filename):
#     print('using read_list_from_file')
    """ Read the zip codes list from the csv file.
        
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')


def set_location(code):
#     print('using get_location')
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    global obs, zlat, zlon
    obs = owm.weather_at_zip_code(f'{code}', 'us')
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return


def current():
#     print('using current')
    ''' Dump the current weather to a json

        :return current: the currently observed weather data
        :type current: dict
    '''
    global obs
    current = json.loads(obs.to_JSON()) # dump all that weather data to a json object
    return(current)


def five_day():
#     print('using five_day')
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code
        :type five_day: dict
    '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    return(json.loads(forecast.to_JSON()))


def get_weather(codes):
#     print('using get_weather')
    ''' Get the weather from the API and load it to the database. 
    
    :param codes: list of zip codes
    :type codes: list of strings
    '''
    client = check_db_access(host, port)
    for code in codes:
        data = {}
        set_location(code)
        data.update({'zipcode': code,
                     'current': current(),
                     'five_day': five_day(),
                    })
        load(data, client)
#         print(f'data in for {code}')
    client.close()
    print('client closed')
    return


def check_db_access(host, port):
#     print('using check_db_access')
    ''' A check that there is write access to the database
    
        :param host: the database host
        :type host: string
        :param port: the database connection port
        :type port: int
    '''
    client = MongoClient(host=host, port=port)
    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print('client open')
    except ConnectionFailure:
        print("Server not available")
        return

    # check the database connections
        # Get a count of the number of databases at the connection (accessible through that port)
        # before attempting to add to it
    db_count_pre = len(client.list_database_names())
        # Add a database and collection
    db = client.test_db
    col = db.test_col

    # Insert something to the db
    post = {'name':'Chuck VanHoff',
           'age':'38',
           'hobby':'gardening'
           }
    col.insert_one(post)

        # Get a count of the databases after adding one
    db_count_post = len(client.list_database_names())

    if db_count_pre-db_count_post>=0:
        print('Your conneciton is flipped up')
    else:
        print('You have write access')

    client.drop_database(db)
    return(client)


def to_json(data, code):
#     print('using to_json')
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable.
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param code: the zip code associated with the data from the list codes
        :type code: sting
    '''
    collection = data
    directory = os.getcwd()
    save_path = path.join(directory, 'Data', f'{code}.json')
    Data = open(save_path, 'a+')
    Data.write(collection)
    Data.close()
    return


def load(data, client):
#     print('using load')
    ''' Load the data to the database if possible, otherwise write to json file. 
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param client: the pymongo client object
        :type client: MongoClient
    '''
    if type(data) == dict:
        try:
            db = client.forcast
            col = db.code
            col.insert_one(data)
            print(f'inserted data for {data["zipcode"]}')
        except DuplicateKeyError:
            client.close()
            print('closed db connection')
            to_json(data, code)
            print('Wrote to json')
    else:
        print('data type is not dict')
        client.close()
        print('closed db connection')
    return

In [3]:
# make_zip_list('nc')
host

'localhost'

In [4]:
filename = 'resources/success_zips.csv'
codes = read_list_from_file(filename)

In [7]:
get_weather(codes)

client open
You have write access
inserted data for 27006
inserted data for 27007
inserted data for 27009
inserted data for 27010
inserted data for 27011
inserted data for 27012
inserted data for 27013
inserted data for 27014
inserted data for 27016
inserted data for 27017
inserted data for 27018
inserted data for 27019
inserted data for 27020
inserted data for 27021
inserted data for 27022
inserted data for 27023
inserted data for 27024
inserted data for 27025
inserted data for 27027
inserted data for 27028
inserted data for 27030
inserted data for 27031
inserted data for 27040
inserted data for 27041
inserted data for 27042
inserted data for 27043
inserted data for 27045
inserted data for 27046
inserted data for 27047
inserted data for 27048
inserted data for 27049
inserted data for 27050
inserted data for 27051
inserted data for 27052
inserted data for 27053
inserted data for 27054
inserted data for 27055
inserted data for 27094
inserted data for 27098
inserted data for 27099
insert

inserted data for 27812
inserted data for 27813
inserted data for 27814
inserted data for 27815
inserted data for 27816
inserted data for 27817
inserted data for 27818
inserted data for 27819
inserted data for 27820
inserted data for 27821
inserted data for 27822
inserted data for 27823
inserted data for 27824
inserted data for 27825
inserted data for 27826
inserted data for 27827
inserted data for 27828
inserted data for 27829
inserted data for 27830
inserted data for 27831
inserted data for 27832
inserted data for 27833
inserted data for 27834
inserted data for 27835
inserted data for 27836
inserted data for 27837
inserted data for 27839
inserted data for 27840
inserted data for 27841
inserted data for 27842
inserted data for 27843
inserted data for 27844
inserted data for 27845
inserted data for 27846
inserted data for 27847
inserted data for 27849
inserted data for 27850
inserted data for 27851
inserted data for 27852
inserted data for 27853
inserted data for 27855
inserted data fo

APICallTimeoutError: Exception in calling OWM Weather API.
Reason: API call timeouted
Caused by: None

In [8]:
code = '27606'

def set_location(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    global obs, zlat, zlon
    obs = owm.weather_at_zip_code(f'{code}', 'us')
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return

In [49]:
def current():
    ''' Dump the current weather to a json

        :return current: the currently observed weather data
        :type current: dict
    '''
    global obs
    current = json.loads(obs.to_JSON()) # dump all that weather data to a json object
    return(current)


In [50]:
# set_location(code)
# # data = current()
# import json
# print(type(data))
# data_json = json.loads(data)
# data_json
# json.loads(data)
# json.loads(obs.to_JSON())
current()

{'reception_time': 1577667812,
 'Location': {'name': 'Raleigh',
  'coordinates': {'lon': -78.71, 'lat': 35.76},
  'ID': 0,
  'country': 'US'},
 'Weather': {'reference_time': 1577667715,
  'sunset_time': 1577657357,
  'sunrise_time': 1577622241,
  'clouds': 90,
  'rain': {'1h': 0.64},
  'snow': {},
  'wind': {'speed': 2.6, 'deg': 180},
  'humidity': 100,
  'pressure': {'press': 1014, 'sea_level': None},
  'temperature': {'temp': 291.52,
   'temp_kf': None,
   'temp_max': 293.15,
   'temp_min': 290.15},
  'status': 'Rain',
  'detailed_status': 'light rain',
  'weather_code': 500,
  'weather_icon_name': '10n',
  'visibility_distance': 16093,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None}}

In [17]:
client = check_db_access(host, port)

def load(data, client):
    ''' Load the data to the database if possible, otherwise write to json file. 
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param client: the pymongo client object
        :type client: MongoClient
    '''
#     if type(data) == dict:
    try:
        db = client.forcast
        col = db.code
        col.insert_one(data)
        print(f'inserted data for {data["zipcode"]}')
    except DuplicateKeyError:
        client.close()
        print('closed db connection')
        to_json(data, code)
        print('Wrote to json')
#     else:
#         print('data type is not dict')
#         client.close()
#         print('closed db connection')
    return

client open
You have write access


In [15]:
load(data, client)

data type is not dict
closed db connection


In [47]:
def five_day():
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code\
        :type five_day: dict
    '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    five_day = forecast.to_JSON()
    return(five_day)

In [67]:
def five_day():
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code\
        :type five_day: dict
    '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    return(json.loads(forecast.to_JSON()))
#     five_day = forecast.to_JSON()
#     return(json.loads(forecast))

In [10]:
host = 'localhost'
port = 27017
check_db_access(host, port)

client open
You have write access


MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [70]:
type(forecast)

dict